In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import pretty_midi
import librosa
import librosa.display
import gc
from sklearn.preprocessing import StandardScaler
import warnings
from collections import Counter
from torch.utils.data import Dataset
import torch



import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset


from sklearn.preprocessing import StandardScaler

from Preprocessing import *
from ExtractGenre import *
from CNN_ExtractGenre import *
import Util as Util

import DatasetLoader as DL

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
InputPath = os.path.realpath('YAMF/genres_original')

GenreMapping = {'metal': 0, 'disco': 1, 'classical': 2, 'hiphop': 3, 'jazz': 4,
          'country': 5, 'pop': 6, 'blues': 7, 'reggae': 8, 'rock': 9}

In [6]:
with open('GenreDataset.pkl', 'rb') as f:
   GenreDataset = pickle.load(f)

In [7]:
CNN_GenreDataset = DiscriminateSongGenre(GenreDataset)

100%|██████████| 13164/13164 [02:01<00:00, 108.26it/s]


2317


In [ ]:
with open('CNN_GenreDataset.pkl', 'rb') as f:
   CNN_GenreDataset = pickle.load(f)

In [ ]:
# with open('CNN_GenreDataset.pkl', 'wb') as f:
#    pickle.dump(CNN_GenreDataset, f)

In [ ]:
Input = os.path.realpath('../Mono_CleanMidi/Eminem/Bad Meets Evil.mid')

mid = mido.MidiFile(Input)
mid

In [ ]:
Input = os.path.realpath('clean_midi/Eminem/Bad Meets Evil.mid')

mid = mido.MidiFile(Input)

for i, track in enumerate(mid.tracks):
   for msg in track:

      if msg.type == 'program_change':
            print(f"Trach {i} Program Change: Channel={msg.channel}, Program={msg.program}")
      else:
          continue

print(mid)

In [ ]:
Input = os.path.realpath('clean_midi/Eminem/Bad Meets Evil.mid')

mid = mido.MidiFile(Input)

for i, track in enumerate(mid.tracks):
   for msg in track:

      if msg.type == 'note_on' and msg.velocity > 0:
            print(msg)
      elif msg.is_meta:
            print(msg)

In [113]:
Input = os.path.realpath('Mono_CleanMidi/Eminem/Stan.mid')

mid = mido.MidiFile(Input)

for i, track in enumerate(mid.tracks):

    has_program_change = any(msg.type == 'program_change' for msg in track)

    if has_program_change:

        Program = [msg.program for msg in track if msg.type == 'program_change'][0]
        Channel = [msg.channel for msg in track if msg.type == 'program_change'][0]
        print(Program, Channel)
        #print(f"\nTrack {i} contains a program_change — processing all messages:")
        #for msg in track:
            #print(msg)

35 0
40 1
25 2
0 9


In [ ]:
def PreProcessing(nDir = 300):

   Dataset = {}

   InputPath = os.path.relpath('Mono_CleanMidi')

   #Selecting a random number of directory
   all_dirs = [d for d in os.listdir(InputPath) if os.path.isdir(os.path.join(InputPath, d))]

   random_dirs = np.random.choice(all_dirs, nDir)

   for dir in tqdm(random_dirs, desc='Preprocessing'):
      DirPath = os.path.join(InputPath, dir)

      if not os.path.isdir(DirPath):
         continue

      #Real all the file in each folder
      for file in os.listdir(DirPath):

         FilePath = os.path.join(DirPath, file)

         #Cleaned monophonic: Some songs are corrupted:
         mid = Func_CorruptedFile(FilePath, file, dir)

         if mid is None:
            continue

         Dataset = ToGeneralInfo(mid, Dataset, file)


   #Remove garbage tracks
   for track in list(Dataset.keys()):
      if len(Dataset[track]['Tempo']) < 20:
         del Dataset[track]

   NormDataset = ReMap_Database(Dataset)
   Dataset = NormDataset

   for track in Dataset.keys():
      Dataset[track]['Bars'] = np.array(Dataset[track]['Bars'])


   return Dataset

In [ ]:
def ToGeneralInfo(mid, Dataset, file):

   Func_Tempo = lambda t: 60_000_000 / t
   TicksPerBeat = mid.ticks_per_beat

   #defining the tempo of file (one for each)
   if len(mid.tracks) > 0:
      InitialTrack = mid.tracks[0]
      for msg in InitialTrack:
         if msg.type == 'set_tempo':

            Tempo = Func_Tempo(msg.tempo)
            break
         else:
            Tempo = 0

   #Loop over all tracks (beside the first --> metadata)
   for track in mid.tracks[1:]:
      #Consider only the tracks that have an instrument in it (remove grabage)
      HasProgramChange = any(msg.type == 'program_change' for msg in track)

      if HasProgramChange:

         Program = [msg.program for msg in track if msg.type == 'program_change'][0]
         Channel = [msg.channel for msg in track if msg.type == 'program_change'][0]

         if Program[0] == 0:
            continue

         #Compute the (128x16) bars matrix for each track
         Bars = ToBars(track, TicksPerBeat)

         if Bars is None:
            continue

         TrackName = f'{file[:-4]}'
         #If there is not the track in the dataset, add it
         if TrackName not in Dataset:               
            Dataset[TrackName] = {
               'Bars': [],
               'Tempo': [], 
               'Program': [], 
               'Channel': [] 
            }
         
         #and add the information to the Dataset dictionary
         Dataset[TrackName]['Bars'].extend(Bars)
         Dataset[TrackName]['Tempo'].extend([int(Tempo)] * len(Bars))
         Dataset[TrackName]['Program'].extend(Program * len(Bars))
         Dataset[TrackName]['Channel'].extend(Channel * len(Bars))

   return Dataset

In [196]:
Input = os.path.realpath('Mono_CleanMidi/Eminem/Stan.mid')

mid = mido.MidiFile(Input)

Dataset = {}
Dataset = ToGeneralInfo(mid, Dataset, 'Stan.mid')

In [138]:
from pympler import asizeof
asizeof.asizeof(Dataset)

#Dataset

10432

In [222]:
test = PreProcessing(nDir = 20)

Preprocessing: 100%|██████████| 20/20 [00:01<00:00, 11.02it/s]


In [ ]:
test

In [223]:
MappedDataset = {}

for key, value in test.items():

   print(key)
   
   for i, prog in enumerate(value['Program']):

      Instrument = Util.InstrumentFamily_Map[prog]

      if Instrument not in MappedDataset:
         MappedDataset[Instrument] = {
            'Bars': [],
            'Tempo': [],
            'Program': [],
            'Channel': []
         }

      MappedDataset[Instrument]['Bars'].extend(value['Bars'][i])
      MappedDataset[Instrument]['Tempo'].append(value['Tempo'][i])
      MappedDataset[Instrument]['Program'].append(prog)
      MappedDataset[Instrument]['Channel'].append(value['Channel'][i])




Too Late for Goodbyes
Gimme Three Steps
Sweet Home Alabama
Free Bird
What's Your Name
I Know a Little
Le temps qui court.1
Symphony No. 2 in B major, Op. 4 B 12 : II. Poco adagio
Crushing Day
Surfing With the Alien
Day at the Beach
The Mighty Turtle Head
Summer Song
Un'emergenza d'amore.1
Strani Amori
Mi Rubi L'anima
Anni miei
Incancellabile
La Solitudine
Il coraggio che non c'e
Mickey
Let's Make the Water Turn Black
Peaches en Regalia
Montana (Whipping Floss)
Bitch
Walk On The Wild Side.1
Loser.1
Where It's At.1
Deadweight
Devil's Haircut.1
Jack-Ass
Readymade
Telstar
Take Me for a Ride
(They Long to Be) Close to You
Please Mister Postman
There's A Kind Of Hush
It's Going to Take Sometime
Top of the World.5
This Masquerade.1
Hurting Each Other
Rainy Days and Mondays
For All We Know.1
It's Going to Take Some Time
Jambalaya (On the Bayou)
Only Yesterday
Yesterday Once More (Reprise).1
Breaking Up Is Hard to Do
Goodbye to Love.1
Just Remember I Love You
Carolina in My Mind
Fire & Rain
Han